In [1]:
from typing import Any
from py3.k_means import PixelValueGenerator, CustomKMeans
import numpy as np
from py3.CNN_model import Cplx_CustomCNN_1D
from py3.Attention_Layer import CustomAttentionLayer, AttentionLayer
from py3.classification import CustomClassifierModel
import tensorflow as tf

import re
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np

In [2]:
# DATA
#_________________________________________________________________
# Load the data
df = pd.read_csv('Prepro_Data/post_processed_data_small20.csv')

def extract_multiple_arrays(string_repr):
    # Remove letters, parentheses, spaces, and unwanted sequences
    string_repr = re.sub('[a-df-zA-DF-Z\(\)\s]', '', string_repr)
    string_repr = re.sub(r',=32', '', string_repr)

    # Find all matches of arrays within the string
    arrays = re.findall(r'\[.*?\]', string_repr)

    # Convert each found array string into a NumPy array
    #np_arrays = [np.array(re.findall(r'[+-]?\d+(?:\.\d+)?', array), dtype=float) for array in arrays]
    np_arrays = [np.array(re.findall(r'[+-]?\d+(?:\.\d+)?(?:e[+-]?\d+)?', array), dtype=float) for array in arrays]
    return np_arrays

df['pixels'] = df['pixels'].apply(extract_multiple_arrays)
print(df.head())

   Unnamed: 0  segment_id  polygon_id  class_id  \
0           0      367854         0.0       3.0   
1           1      367855         0.0       3.0   
2           2      367856         0.0       3.0   
3           3      369130         0.0       3.0   
4           4      363215         1.0       1.0   

                                              pixels  
0  [[0.44984567, 0.49691358, 0.40354937, 0.622685...  
1  [[0.55632716, 0.5578704, 0.50848764, 0.7075617...  
2  [[0.53780866, 0.5671296, 0.4699074, 0.69058645...  
3  [[0.5231481, 0.5532407, 0.44984567, 0.6705247,...  
4  [[0.38194445, 0.43904322, 0.24845679, 0.493055...  


In [3]:
# ALGORITHM
#_________________________________________________________________
# K-means
stacked_arrays = []

# Stack the arrays for each cell
for i, cell_pixels in enumerate(df['pixels']):
    stacked_array = np.vstack(cell_pixels)
    stacked_arrays.append(stacked_array)
        
n_clusters=2
custom_kmeans = CustomKMeans(n_clusters=n_clusters)

clustered_data = []
clustered_labels = []
for i in range (len(stacked_arrays)):
    custom_kmeans.fit(stacked_arrays[i])
    clustered_data.append(custom_kmeans.get_cluster_centers())
    clustered_labels.append(custom_kmeans.get_cluster_labels())
    
clusters = np.array(clustered_data)
print(clusters.shape)

/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/si

(4795, 2, 73, 10)


/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/si

In [11]:
class Custom_Model(tf.keras.Model):
    '''
    This class defines the model architecture.
    '''
    
    def __init__(self):
        super(Custom_Model, self).__init__()
        self.enc = Cplx_CustomCNN_1D()
        self.attn = AttentionLayer()
        self.classifier = CustomClassifierModel(num_classes=8, fc_units=64) 

    def call(self, inputs):
        input_list = tf.unstack(inputs,axis=1) # from tensor to list
        intermediate = [self.enc(input) for input in input_list]
        intermediate = tf.stack(intermediate,axis=1)        
        emb, alphas = self.attn(intermediate)
        
        return self.classifier(emb)
    
    def get_alphas(self, inputs):
        input_list = tf.unstack(inputs,axis=1) # from tensor to list
        intermediate = [self.enc(input) for input in input_list]
        intermediate = tf.stack(intermediate,axis=1)        
        _, alphas = self.attn(intermediate)
        
        return alphas


In [12]:
model = Custom_Model()
input_shape = (2, 73, 10)  # Define your input shape

# Create a dummy input tensor for model initialization
dummy_input = tf.random.normal((1,) + input_shape)

# Pass the dummy input through the model to initialize the layers
model(dummy_input)


<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[0.12562631, 0.12323729, 0.12433685, 0.12698229, 0.12312446,
        0.12809266, 0.12254906, 0.12605108]], dtype=float32)>

In [13]:
# Load the weights
SAVE_PATH = 'saved_models/'
model.load_weights(SAVE_PATH + 'model_weights_100_epochs.h5')

In [14]:
predictions = model.predict(clusters)

df_inferring = df.iloc[:, :4]
# Get the class with the highest probability
df_inferring['class_id'] = np.argmax(predictions, axis=1) + 1
df_inferring['class_id'] = df_inferring['class_id'].astype('int8')

# Pixels' cluster
# Assuming you have 'clustered_labels' available
df_inferring['pixels_cluster'] = clustered_labels

# Alphas
alphas = model.get_alphas(clusters)
alphas = np.array(alphas)

df_inferring['alphas'] = alphas.tolist()
print(df_inferring.head())

# Save the results
df_inferring.to_csv('Inference_results.csv', index=False)

2023-12-17 01:26:47.839272: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


150/150 [==============================] - 7s 34ms/step
   Unnamed: 0  segment_id  polygon_id  class_id  \
0           0      367854         0.0         3   
1           1      367855         0.0         3   
2           2      367856         0.0         3   
3           3      369130         0.0         3   
4           4      363215         1.0         3   

                                      pixels_cluster  \
0   [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]   
1   [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]   
2   [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]   
3         [0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1]   
4  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                      alphas  
0  [0.5192344188690186, 0.48076558113098145]  
1    [0.4495514929294586, 0.550448477268219]  
2  [0.6534169912338257, 0.34658294916152954]  
3  [0.5717190504074097, 0.42828094959259033]  
4   [0.5950378179550171, 0.4049621820449829]  
